In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
output_dir = "huggingface_training"

## Training

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, device_map=device)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    num_train_epochs=3,
    learning_rate=1e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

## Inference

In [ ]:
untrained_model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, device_map=device)
trained_model = AutoModelForSequenceClassification.from_pretrained(output_dir, num_labels=5, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [ ]:
text = "dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
encoding = tokenizer(text, return_tensors="pt").to(device)

untrained_output = untrained_model(**encoding)
untrained_pred = untrained_output.logits.argmax(-1)

trained_output = trained_model(**encoding)
trained_pred = trained_output.logits.argmax(-1)

untrained_pred, trained_pred